In [1]:
import pandas as pd
from mlfinlab.optimal_mean_reversion import OrnsteinUhlenbeck
from datetime import date
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
backtest_data = pd.read_csv("../backtest_data_entries_exits_2018.csv")
#Final price of each day
backtest_data["datetime"] = pd.to_datetime(backtest_data["datetime"])
backtest_data["dayPeriod"] = backtest_data["datetime"].dt.to_period("D")
backtest_daily = backtest_data.groupby(backtest_data["dayPeriod"]).apply(lambda x: x.iloc[[-1]])


slopes = pd.read_csv("../hedge_ratios_2018.csv")
slopes["datetime"] = backtest_data["datetime"]
slopes["dayPeriod"] = slopes["datetime"].dt.to_period("D")
slopes_daily = slopes.groupby(slopes["dayPeriod"]).apply(lambda x: x.iloc[[-1]])

print(backtest_daily.shape, slopes_daily.shape)

(251, 13) (251, 12)


In [3]:
print(slopes_daily.columns)
slopes_daily.rename(columns={"hasHR":"hasclose", "ttwoHR":"ttwoclose", "ctasHR":"ctasclose", 
                             "aaplHR": "aaplclose", "algnHR":"algnclose", "sbuxHR":"sbuxclose", "payxHR":"payxclose", 
                             "alxnHR":"alxnclose"}, inplace=True)
print(slopes_daily.columns)

Index(['Unnamed: 0', 'hasHR', 'aaplHR', 'ttwoHR', 'sbuxHR', 'ctasHR', 'alxnHR',
       'algnHR', 'payxHR', 'intercept', 'datetime', 'dayPeriod'],
      dtype='object')
Index(['Unnamed: 0', 'hasclose', 'aaplclose', 'ttwoclose', 'sbuxclose',
       'ctasclose', 'alxnclose', 'algnclose', 'payxclose', 'intercept',
       'datetime', 'dayPeriod'],
      dtype='object')


In [4]:
qqq = backtest_daily["qqqclose"].apply(np.log)
tickers = ["hasclose", "aaplclose", "ttwoclose", "sbuxclose", "ctasclose", "alxnclose", "algnclose", "payxclose"]
synth = backtest_daily[tickers]
start = date(2018, 1, 2)
end = date(2018, 12, 31)
dt = 1.0 / (end-start).days

# get historical market data
#gld_price = gld.history(period="100d")
#slv_price = slv.history(period="100d")
#ts_gld = gld_price["Close"]
#ts_slv = slv_price["Close"]

In [5]:
def construct_synth_asset(synth_comps, weights, assets):
    weighted_prices = weights[assets].multiply(np.log(synth_comps[assets]))
    weighted_prices['intercept'] = weights['intercept']
    synth_asset = weighted_prices.sum(1)
    return synth_asset


In [6]:
synth_asset = construct_synth_asset(synth, slopes_daily, tickers)
synth_asset.rename("synth_close", inplace = True)
train_df = pd.concat([qqq, synth_asset], axis = 1)

In [7]:
train_df

,,qqqclose,synth_close
dayPeriod,,,
2018-01-02,353,5.065439,5.065067
2018-01-03,713,5.075424,5.075265
2018-01-04,1073,5.077421,5.078937
2018-01-05,1431,5.086732,5.087051
2018-01-08,1788,5.090924,5.091512
...,...,...,...
2018-12-24,86861,4.964731,4.965863
2018-12-26,87230,5.018537,5.018316
2018-12-27,87598,5.026410,5.027863


In [8]:
ou_model = OrnsteinUhlenbeck()
ou_model.fit(train_df, data_frequency = "D", discount_rate = [0.05, 0.05], 
             transaction_cost = [0, 0])
print(ou_model.check_fit())
print("Model description:\n",ou_model.description())

              Theta          Mu     Sigma  Max log-likelihood
empirical -0.000075  133.610633  0.003304            7.299982
simulated -0.000052  133.076957  0.003968            7.064522
Model description:
 training period              [(2018-01-02, 353), (2018-12-31, 88326)]
long-term mean                                           -7.52178e-05
speed of reversion                                            133.611
volatility                                                 0.00330424
fitting error                                                0.245278
discount rates                                           [0.05, 0.05]
transaction costs                                              [0, 0]
stop-loss level                                                  None
beta                                                                1
optimal entry level                                                 0
optimal liquidation level                                           0
dtype: object


C:\Users\liuvi\anaconda3\envs\quant\lib\site-packages\mlfinlab\optimal_mean_reversion\ou_model.py:572: RuntimeWarning: overflow encountered in exp
  * np.exp(np.sqrt(2 * self.mu / self.sigma_square)
C:\Users\liuvi\anaconda3\envs\quant\lib\site-packages\mlfinlab\optimal_mean_reversion\ou_model.py:541: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  calculated_f = quad(f_func, 0, np.inf)[0]
C:\Users\liuvi\anaconda3\envs\quant\lib\site-packages\mlfinlab\optimal_mean_reversion\ou_model.py:541: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  calculated_f = quad(f_func, 0, np.inf)[0]
C:\Users\liuvi\anaconda3\envs\quant\lib\site-packages\mlfinlab\optimal_mean_reversion\ou_model.py:541: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  calculated_f = quad(f_func, 0, np.inf)[0]
C:\Users\liuvi\anaconda3\envs\quant\lib\site-packages\mlfinlab\optimal_mean_reversion\ou_model.py:535: RuntimeWarning: overflo